<a href="https://colab.research.google.com/github/Anshuman-37/Twitter_Sentiment_Analysis/blob/main/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading the data from Kaggle

In [5]:
## Prerequisites
#!pip install kaggle --upgrade
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [9]:
## Downloading data command
!kaggle datasets download -d manchunhui/us-election-2020-tweets

 97% 344M/353M [00:02<00:00, 154MB/s]
100% 353M/353M [00:02<00:00, 169MB/s]


In [11]:
!mkdir content/data
!unzip us-election-2020-tweets.zip -d data

mkdir: cannot create directory ‘content/data’: No such file or directory
Archive:  us-election-2020-tweets.zip
  inflating: data/hashtag_donaldtrump.csv  
  inflating: data/hashtag_joebiden.csv  


## Download Pyspark

In [3]:
!pip install pyspark 

## Import Header Files

In [1]:
## Python 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 


## Spark
import pyspark.sql.functions as F
from pyspark.sql import Row
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType , FloatType

# Create Spark Session
spark = SparkSession.builder.master("local[1]").appName('Twitter Sentiment Analysis').getOrCreate()

## Reading the Data

#### Defining the Schema

In [6]:

# Created the schema that we need 
# Fucking Brute Force I dont like it
mySchema = StructType([ StructField('created_at', StringType(), True)\
                       ,StructField('tweet_id', FloatType(), True)\
                       ,StructField('tweet',StringType(),True)\
                       ,StructField('likes',FloatType(),True)\
                       ,StructField('retweet_count',FloatType(),True)\
                       ,StructField('source',StringType(),True)\
                       ,StructField('user_id',FloatType(),True)\
                       ,StructField('user_name',StringType(),True)\
                       ,StructField('user_screen_name',StringType(),True)\
                       ,StructField('user_description',StringType(),True)\
                       ,StructField('user_join_date',StringType(),True)\
                       ,StructField('user_followers_count',FloatType(),True)\
                       ,StructField('user_location',StringType(),True)\
                       ,StructField('lat',FloatType(),True)\
                       ,StructField('long',FloatType(),True)\
                       ,StructField('city',StringType(),True)\
                       ,StructField('country',StringType(),True)\
                       ,StructField('continent',StringType(),True)\
                       ,StructField('state',StringType(),True)\
                       ,StructField('state_code',StringType(),True)\
                       ,StructField('collected_at',StringType(),True)\
                       ])

In [7]:
# Creating spark data frame
# This shit was so hard to do it was not reading multiline data at first it killed me
# Now have to change the schema to different data type

trump_df = spark.read.option('multiline',True)\
                     .option('header', 'true')\
                     .option('lineterminator',"\n")\
                     .option('inferSchema', "true")\
                     .schema(mySchema)\
                     .csv('/content/data/hashtag_donaldtrump.csv')
trump_df.cache()
print('Trump DataFrame Schema')
trump_df.printSchema()


biden_df = spark.read.option('multiline',True)\
                     .option('header', 'true')\
                     .option('lineterminator',"\n")\
                     .option('inferSchema', "true")\
                     .schema(mySchema)\
                     .csv('/content/data/hashtag_joebiden.csv')

biden_df.cache()
print('Biden DataFrame Schema')
biden_df.printSchema()


Trump DataFrame Schema
root
 |-- created_at: string (nullable = true)
 |-- tweet_id: float (nullable = true)
 |-- tweet: string (nullable = true)
 |-- likes: float (nullable = true)
 |-- retweet_count: float (nullable = true)
 |-- source: string (nullable = true)
 |-- user_id: float (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_screen_name: string (nullable = true)
 |-- user_description: string (nullable = true)
 |-- user_join_date: string (nullable = true)
 |-- user_followers_count: float (nullable = true)
 |-- user_location: string (nullable = true)
 |-- lat: float (nullable = true)
 |-- long: float (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- state: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- collected_at: string (nullable = true)

Biden DataFrame Schema
root
 |-- created_at: string (nullable = true)
 |-- tweet_id: float (nullable = true)
 

In [8]:
# Took me 35 seconds to run
trump_df.show(5)

+-------------------+-------------+--------------------+-----+-------------+------------------+------------+--------------------+----------------+--------------------+-------------------+--------------------+--------------------+---------+-----------+----------+--------------------+-------------+--------------------+----------+--------------------+
|         created_at|     tweet_id|               tweet|likes|retweet_count|            source|     user_id|           user_name|user_screen_name|    user_description|     user_join_date|user_followers_count|       user_location|      lat|       long|      city|             country|    continent|               state|state_code|        collected_at|
+-------------------+-------------+--------------------+-----+-------------+------------------+------------+--------------------+----------------+--------------------+-------------------+--------------------+--------------------+---------+-----------+----------+--------------------+-------------+-

In [9]:
biden_df.show(5)

+-------------------+-------------+--------------------+-----+-------------+------------------+-------------+------------------+----------------+--------------------+-------------------+--------------------+--------------------+---------+----------+----+--------------------+-------------+----------+----------+--------------------+
|         created_at|     tweet_id|               tweet|likes|retweet_count|            source|      user_id|         user_name|user_screen_name|    user_description|     user_join_date|user_followers_count|       user_location|      lat|      long|city|             country|    continent|     state|state_code|        collected_at|
+-------------------+-------------+--------------------+-----+-------------+------------------+-------------+------------------+----------------+--------------------+-------------------+--------------------+--------------------+---------+----------+----+--------------------+-------------+----------+----------+--------------------+
|

## IDEA- 
1. Change Data Type 

> ~~Everything is a string type first convert it to the required type and change the schema~~ 

~~<em>Answer -  I think I will brute force on this one getting all the column names and changing the schema</em>~~

Done

2. Data visualization


> We need to visualize the data and check for imbalance. To do that we need to concatinate the data

3. Data cleaning

> Haven't thought about it in detail but we need to do normal nltk steps to do this 

4. Analysis

> Think of using a library or we need to do it on our own ? 

> Do we need to use vector assembler and the other shit as well to do this. I am not sure..










